<h2>Importing librarys and data</h2>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

/Users/sse/opt/anaconda3/envs/py3.10/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_yout = pd.read_csv("../input/youtube-new/USvideos.csv")

In [8]:
df = df_yout
intended_df_size_in_MB = 256
factor = intended_df_size_in_MB*(2**20)/df.memory_usage(index=True).sum()
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
df_yout = df
df_yout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1931189 entries, 0 to 1931188
Data columns (total 20 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   video_id                object 
 1   trending_date           object 
 2   title                   object 
 3   channel_title           object 
 4   category_id             int64  
 5   publish_time            object 
 6   tags                    object 
 7   views                   int64  
 8   likes                   int64  
 9   dislikes                int64  
 10  comment_count           int64  
 11  thumbnail_link          object 
 12  comments_disabled       bool   
 13  ratings_disabled        bool   
 14  video_error_or_removed  bool   
 15  description             object 
 16  likes_log               float64
 17  views_log               float64
 18  dislikes_log            float64
 19  comment_log             float64
dtypes: bool(3), float64(4), int64(5), object(8)
memory usage: 256.0+ MB


In [4]:
#Looking some information of the data
print(df_yout.shape)
print(df_yout.nunique())

(2497889, 16)
video_id                   6351
trending_date               205
title                      6455
channel_title              2207
category_id                  16
publish_time               6269
tags                       6055
views                     40478
likes                     29850
dislikes                   8516
comment_count             13773
thumbnail_link             6352
comments_disabled             2
ratings_disabled              2
video_error_or_removed        2
description                6901
dtype: int64


In [5]:
#Looking for Nulls and type of our data
df_yout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2497889 entries, 0 to 2497888
Data columns (total 16 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   video_id                object
 1   trending_date           object
 2   title                   object
 3   channel_title           object
 4   category_id             int64 
 5   publish_time            object
 6   tags                    object
 7   views                   int64 
 8   likes                   int64 
 9   dislikes                int64 
 10  comment_count           int64 
 11  thumbnail_link          object
 12  comments_disabled       bool  
 13  ratings_disabled        bool  
 14  video_error_or_removed  bool  
 15  description             object
dtypes: bool(3), int64(5), object(8)
memory usage: 254.9+ MB


In [6]:
df_yout.head(n=2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."


<h2>Let's start looking if Views, likes, dislikes and comment counts have a normal distribuition</h2>

In [7]:
df_yout['likes_log'] = np.log(df_yout['likes'] + 1)
df_yout['views_log'] = np.log(df_yout['views'] + 1)
df_yout['dislikes_log'] = np.log(df_yout['dislikes'] + 1)
df_yout['comment_log'] = np.log(df_yout['comment_count'] + 1)

# plt.figure(figsize = (12,6))

# plt.subplot(221)
# g1 = sns.distplot(df_yout['views_log'])
# g1.set_title("VIEWS LOG DISTRIBUITION", fontsize=16)

# plt.subplot(224)
# g2 = sns.distplot(df_yout['likes_log'],color='green')
# g2.set_title('LIKES LOG DISTRIBUITION', fontsize=16)

# plt.subplot(223)
# g3 = sns.distplot(df_yout['dislikes_log'], color='r')
# g3.set_title("DISLIKES LOG DISTRIBUITION", fontsize=16)

# plt.subplot(222)
# g4 = sns.distplot(df_yout['comment_log'])
# g4.set_title("COMMENTS LOG DISTRIBUITION", fontsize=16)

# plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 0.9)

# plt.show()

In [ ]:
print("Views quantiles")
print(df_yout['views'].quantile([.01,.25,.5,.75,.99]))
print("")
print("Likes quantiles")
print(df_yout['likes'].quantile([.01,.25,.5,.75,.99]))
print("")
print("Dislikes quantiles")
print(df_yout['dislikes'].quantile([.01,.25,.5,.75,.99]))
print("")
print("Comment quantiles")
print(df_yout['comment_count'].quantile([.01,.25,.5,.75,.99]))

In [ ]:
df_yout['category_name'] = np.nan

df_yout.loc[(df_yout["category_id"] == 1),"category_name"] = 'Film and Animation'
df_yout.loc[(df_yout["category_id"] == 2),"category_name"] = 'Cars and Vehicles'
df_yout.loc[(df_yout["category_id"] == 10),"category_name"] = 'Music'
df_yout.loc[(df_yout["category_id"] == 15),"category_name"] = 'Pets and Animals'
df_yout.loc[(df_yout["category_id"] == 17),"category_name"] = 'Sport'
df_yout.loc[(df_yout["category_id"] == 19),"category_name"] = 'Travel and Events'
df_yout.loc[(df_yout["category_id"] == 20),"category_name"] = 'Gaming'
df_yout.loc[(df_yout["category_id"] == 22),"category_name"] = 'People and Blogs'
df_yout.loc[(df_yout["category_id"] == 23),"category_name"] = 'Comedy'
df_yout.loc[(df_yout["category_id"] == 24),"category_name"] = 'Entertainment'
df_yout.loc[(df_yout["category_id"] == 25),"category_name"] = 'News and Politics'
df_yout.loc[(df_yout["category_id"] == 26),"category_name"] = 'How to and Style'
df_yout.loc[(df_yout["category_id"] == 27),"category_name"] = 'Education'
df_yout.loc[(df_yout["category_id"] == 28),"category_name"] = 'Science and Technology'
df_yout.loc[(df_yout["category_id"] == 29),"category_name"] = 'Non Profits and Activism'
df_yout.loc[(df_yout["category_id"] == 25),"category_name"] = 'News & Politics'

<h2>Looking the Views, likes, dislikes distribuition by category_name's</h2>

In [ ]:
print("Category Name count")
print(df_yout.category_name.value_counts()[:5])

# plt.figure(figsize = (14,9))

# plt.subplot(211)
# g = sns.countplot('category_name', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Counting the Video Category's ", fontsize=15)
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Count", fontsize=12)

# plt.subplot(212)
# g1 = sns.boxplot(x='category_name', y='views_log', data=df_yout, palette="Set1")
# g1.set_xticklabels(g.get_xticklabels(),rotation=45)
# g1.set_title("Views Distribuition by Category Names", fontsize=20)
# g1.set_xlabel("", fontsize=15)
# g1.set_ylabel("Views(log)", fontsize=15)

# plt.subplots_adjust(hspace = 0.9, top = 0.9)

# plt.show()

Significant number of politics and news videos.

The unique difference is in Non Profits and Activism videos that have less views than the "normal" videos


<h2>Looking the Views, likes, dislikes distribuition by category_name's</h2>

In [ ]:
# plt.figure(figsize = (14,6))

# g = sns.boxplot(x='category_name', y='likes_log', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Likes Distribuition by Category Names ", fontsize=15)
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Likes(log)", fontsize=12)
# plt.show()

News and Politics have a the same of Non Profits of engagement likes

In [ ]:
# plt.figure(figsize = (14,6))

# g = sns.boxplot(x='category_name', y='dislikes_log', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Dislikes distribuition by Category's", fontsize=15)
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Dislikes(log)", fontsize=12)
# plt.show()

In [ ]:
# plt.figure(figsize = (14,6))

# g = sns.boxplot(x='category_name', y='comment_log', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Comments Distribuition by Category Names", fontsize=15)
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Comments Count(log)", fontsize=12)

# plt.show()

<h1>Engagement Features </h1>

I will create some new variables to us analyse the percent of likes, dislikes, comment by each category to discover what category have the highest engagement

In [ ]:
df_yout['like_rate'] =  df_yout ['likes'] / df_yout['views'] * 100
df_yout['dislike_rate'] =  df_yout ['dislikes'] / df_yout['views'] * 100
df_yout['comment_rate'] =  df_yout ['comment_count'] / df_yout['views'] * 100

In [ ]:
# plt.figure(figsize = (9,6))

# g1 = sns.distplot(df_yout['dislike_rate'], color='red',hist=False, label="Dislike")
# g1 = sns.distplot(df_yout['like_rate'], color='green',hist=False, label="Like")
# g1 = sns.distplot(df_yout['comment_rate'],hist=False,label="Comment")
# g1.set_title('CONVERT RATE DISTRIBUITION', fontsize=16)
# plt.legend()
# plt.show()

Very meaningful distribuition! Like x Dislikes rate have interesting differences

In [ ]:
# plt.figure(figsize = (12,18))

# plt.subplot(311)
# g= sns.boxplot(x='category_name',y='like_rate',  data=df_yout)
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("LIKE RATE DISTRIBUITIONS", fontsize=15)
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Like rate", fontsize=12)

# plt.subplot(312)
# g1= sns.boxplot(y='dislike_rate', x='category_name', data=df_yout)
# g1.set_xticklabels(g.get_xticklabels(),rotation=45)
# g1.set_title("DISLIKE RATE DISTRIBUITIONS", fontsize=15)
# g1.set_xlabel("", fontsize=12)
# g1.set_ylabel("Dislike rate", fontsize=12)

# plt.subplot(313)
# g2= sns.boxplot(y='comment_rate', x='category_name', data=df_yout)
# g2.set_xticklabels(g.get_xticklabels(),rotation=45)
# g2.set_title("COMMENT RATE BY CATEGORY NAME", fontsize=15)
# g2.set_xlabel("Category Names", fontsize=12)
# g2.set_ylabel("Comment Rate", fontsize=12)

# plt.subplots_adjust(wspace = 0.2, hspace = 0.5,top = 0.9)

# plt.show()

We can see that the Dislike rate is very low in almost all categories but some outliers in 'People and Blogs' and News & Politics that we can might can consider "Normal"

The mean of ike distribuition is less than 5% but in music we have a very interesting pattern of like rate... The music category have the highest engagement rate.

And at the Comment rate we can see the how-to category with the highest rates of commments. Interesting because isn't politics LOL

<h2>Looking through the anothers categoricals variables </h2>
We have the variables:
- Comments Disable
- Rating disable
- Video error


In [ ]:
# plt.figure(figsize = (14,8))
# plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 0.9)

# plt.subplot(2,2,1)
# g = sns.countplot(x='comments_disabled', data=df_yout)
# g.set_title("Comments Disabled", fontsize=16)

# plt.subplot(2,2,2)
# g1 = sns.countplot(x='ratings_disabled', data=df_yout)
# g1.set_title("Rating Disabled", fontsize=16)

# plt.subplot(2,2,3)
# g2 = sns.countplot(x='video_error_or_removed', data=df_yout)
# g2.set_title("Video Error or Removed", fontsize=16)
# plt.show()

<h2>Let's look the kde of this variables separated to deep understand them</h2>

In [ ]:
# plt.figure(figsize = (12,10))

# plt.subplot(221)
_ = df_yout[df_yout['comments_disabled'] == True]['views_log']
_ = df_yout[df_yout['ratings_disabled'] == True]['views_log']
_ = df_yout[df_yout['video_error_or_removed'] == True]['views_log']

# plt.subplot(222)
_ = df_yout[df_yout['comments_disabled'] == True]['comment_log']
_ = df_yout[df_yout['ratings_disabled'] == True]['comment_log']
_ = df_yout[df_yout['video_error_or_removed'] == True]['comment_log']

_ = df_yout[df_yout['comments_disabled'] == True]['dislikes_log']
_ = df_yout[df_yout['ratings_disabled'] == True]['dislikes_log']
_ = df_yout[df_yout['video_error_or_removed'] == True]['dislikes_log']

_ = df_yout[df_yout['comments_disabled'] == True]['likes_log']
_ = df_yout[df_yout['ratings_disabled'] == True]['likes_log']
_ = df_yout[df_yout['video_error_or_removed'] == True]['likes_log']

Removed videos no have some views until be moreved... <br>Rating disable and Comments disable have the same distribuition 

<h2>Analysing the Correlation Matrix of the numerical variables</h2>

In [ ]:
# plt.figure(figsize = (10,8))

#Let's verify the correlation of each value
_ = df_yout[['like_rate', 'dislike_rate', 'comment_rate', 'comment_log',
         'views_log','likes_log','dislikes_log', "category_name"]].corr()

<h2>I will start creating some features from texts using Title and Tags columns</h2>

In [ ]:
#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   

In [ ]:
#Setting the stopwords
eng_stopwords = set(stopwords.words("english"))


In [ ]:
#Word count in each comment:
df_yout['count_word']=df_yout["title"].apply(lambda x: len(str(x).split()))
df_yout['count_word_tags']=df_yout["tags"].apply(lambda x: len(str(x).split()))

#Unique word count
df_yout['count_unique_word']=df_yout["title"].apply(lambda x: len(set(str(x).split())))
df_yout['count_unique_word_tags']=df_yout["tags"].apply(lambda x: len(set(str(x).split())))

#Letter count
df_yout['count_letters']=df_yout["title"].apply(lambda x: len(str(x)))
df_yout['count_letters_tags']=df_yout["tags"].apply(lambda x: len(str(x)))

#punctuation count
df_yout["count_punctuations"] =df_yout["title"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_yout["count_punctuations_tags"] =df_yout["tags"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

#upper case words count
df_yout["count_words_upper"] = df_yout["title"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_yout["count_words_upper_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

#title case words count
df_yout["count_words_title"] = df_yout["title"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_yout["count_words_title_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

#Number of stopwords
df_yout["count_stopwords"] = df_yout["title"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_yout["count_stopwords_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

#Average length of the words
df_yout["mean_word_len"] = df_yout["title"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_yout["mean_word_len_tags"] = df_yout["tags"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
#derived features
#Word count percent in each comment:
df_yout['word_unique_percent']=df_yout['count_unique_word']*100/df_yout['count_word']
df_yout['word_unique_percent_tags']=df_yout['count_unique_word_tags']*100/df_yout['count_word_tags']

#Punct percent in each comment:
df_yout['punct_percent']=df_yout['count_punctuations']*100/df_yout['count_word']
df_yout['punct_percent_tags']=df_yout['count_punctuations_tags']*100/df_yout['count_word_tags']

In [ ]:
# plt.figure(figsize = (12,18))

# plt.subplot(421)
# g1 = sns.distplot(df_yout['count_word'], 
#                   hist=False, label='Text')
# g1 = sns.distplot(df_yout['count_word_tags'], 
#                   hist=False, label='Tags')
# g1.set_title("COUNT WORDS DISTRIBUITION", fontsize=16)

# plt.subplot(422)
# g2 = sns.distplot(df_yout['count_unique_word'],
#                   hist=False, label='Text')
# g2 = sns.distplot(df_yout['count_unique_word_tags'], 
#                   hist=False, label='Tags')
# g2.set_title("COUNT UNIQUE DISTRIBUITION", fontsize=16)

# plt.subplot(423)
# g3 = sns.distplot(df_yout['count_letters'], 
#                   hist=False, label='Text')
# g3 = sns.distplot(df_yout['count_letters_tags'], 
#                   hist=False, label='Tags')
# g3.set_title("COUNT LETTERS DISTRIBUITION", fontsize=16)

# plt.subplot(424)
# g4 = sns.distplot(df_yout["count_punctuations"], 
#                   hist=False, label='Text')
# g4 = sns.distplot(df_yout["count_punctuations_tags"], 
#                   hist=False, label='Tags')
# g4.set_xlim([-2,50])
# g4.set_title('COUNT PONCTUATIONS DISTRIBUITION', fontsize=16)

# plt.subplot(425)
# g5 = sns.distplot(df_yout["count_words_upper"] , 
#                   hist=False, label='Text')
# g5 = sns.distplot(df_yout["count_words_upper_tags"] , 
#                   hist=False, label='Tags')
# g5.set_title('COUNT WORDS UPPER DISTRIBUITION', fontsize=16)

# plt.subplot(426)
# g6 = sns.distplot(df_yout["count_words_title"], 
#                   hist=False, label='Text')
# g6 = sns.distplot(df_yout["count_words_title_tags"], 
#                   hist=False, label='Tags')
# g6.set_title('WORDS DISTRIBUITION', fontsize=16)

# plt.subplot(427)
# g7 = sns.distplot(df_yout["count_stopwords"], 
#                   hist=False, label='Title')
# g7 = sns.distplot(df_yout["count_stopwords_tags"], 
#                   hist=False, label='Tags')
# g7.set_title('STOPWORDS DISTRIBUITION', fontsize=16)

# plt.subplot(428)
# g8 = sns.distplot(df_yout["mean_word_len"], 
#                   hist=False, label='Text')
# g8 = sns.distplot(df_yout["mean_word_len_tags"], 
#                   hist=False, label='Tags')
# g8.set_xlim([-2,100])
# g8.set_title('MEAN WORD LEN DISTRIBUITION', fontsize=16)

# plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 0.9)
# plt.legend()
# plt.show()

<h2>Have the Punctuations title and tags any relation with views, likes, dislikes comments? </h2>

In [ ]:
# plt.figure(figsize = (12,8))

# plt.subplot(221)
# g=sns.boxplot(x='count_punctuations', y='views_log',data=df_yout)
# g.set_title("Vews by Ponctuations")
# g.set_xlabel("Numer of Punctuations")
# g.set_ylabel("Vews log")

# plt.subplot(222)
# g1 = sns.boxplot(x='count_punctuations', y='likes_log',data=df_yout)
# g1.set_title("Likes by Ponctuations")
# g1.set_xlabel("Numer of Punctuations")
# g1.set_ylabel("Likes log")

# plt.subplot(223)
# g2 = sns.boxplot(x='count_punctuations', y='dislikes_log',data=df_yout)
# g2.set_title("Dislikes by Ponctuations")
# g2.set_xlabel("Numer of Punctuations")
# g2.set_ylabel("Dislikes log")

# plt.subplot(224)
# g3 = sns.boxplot(x='count_punctuations', y='comment_log',data=df_yout)
# g3.set_title("Comments by Ponctuations")
# g3.set_xlabel("Numer of Punctuations")
# g3.set_ylabel("Comments log")

# plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 0.9)

# plt.show()

<h2>The same view using Tags punctuations values</h2>

In [ ]:
_=df_yout[df_yout['count_punctuations_tags'] < 20]

_=df_yout[df_yout['count_punctuations_tags'] < 20]

_ =df_yout[df_yout['count_punctuations_tags'] < 20]
_ =df_yout[df_yout['count_punctuations_tags'] < 20]

<h2>Let's verify the correlation of title and tags values with views</h2>
- exploring the Correlation matrix to see if have any interesting relation

In [ ]:
_ = df_yout[['count_word', 'count_unique_word','count_letters',
                     "count_punctuations","count_words_upper", "count_words_title", 
                     "count_stopwords","mean_word_len", 
                     'views_log', 'likes_log','dislikes_log','comment_log',
                     'ratings_disabled', 'comments_disabled', 'video_error_or_removed']].corr()

<h1>Visualizing  the Word Cloud of Tagged videos </h1>
- Ratings Disable 
- Comments Disable
- Video Error or Removed

In [ ]:
# mpl.rcParams['font.size']= 15              
# mpl.rcParams['savefig.dpi']= 100         
# mpl.rcParams['figure.subplot.bottom']= .1 

<h3>Rating Disabled</h3>

In [ ]:
# plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)

# wordcloud = WordCloud(
#                           background_color='black',
#                           stopwords=stopwords,
#                           max_words=1000,
#                           max_font_size=120, 
#                           random_state=42
#                          ).generate(str(df_yout['title']))

# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.title("WORD CLOUD - TITLES")
# plt.axis('off')
# plt.show()

<h3>Comments Disabled</h3>

<h2>General Title Word Cloud</h2>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)
# wordcloud = WordCloud(
#                           background_color='black',
#                           stopwords=stopwords,
#                           max_words=150,
#                           max_font_size=40, 
#                           random_state=42
#                          ).generate(str(df_yout['title']))

# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.title("WORD CLOUD - DESCRIPTION")
# plt.axis('off')
# plt.show()

<h1>Visualizing the WordCloud of Description</h1>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)

newStopWords= ['https', 'youtube', 'VIDEO','youtu','CHANNEL', 'WATCH']

stopwords.update(newStopWords)

# wordcloud = WordCloud(
#                           background_color='black',
#                           stopwords=stopwords,
#                           max_words=1200,
#                           max_font_size=80, 
#                           random_state=42
#                          ).generate(str(df_yout['description']))

# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.title("WORD CLOUD - DESCRIPTION")
# plt.axis('off')
# plt.show()


<h1>Analysing the Wordcloud of Tags</h1>

In [ ]:
# plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)
# wordcloud = WordCloud(
#                           background_color='black',
#                           stopwords=stopwords,
#                           max_words=1200,
#                           max_font_size=120, 
#                           random_state=42
#                          ).generate(str(df_yout['tags']))
# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.title("WORD CLOUD - TAGS")
# plt.axis('off')
# plt.show()

I will continue implementing this analysis because I want learn about text data! <br>

Thank you very much for your view and <b> votes up</b> my kernel 

In [ ]:
df_yout['publish_time'] = pd.to_datetime(df_yout['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
df_yout['month'] = df_yout['publish_time'].dt.month

print("Category Name count")
print(df_yout['month'].value_counts()[:5])

# plt.figure(figsize = (14,9))

# plt.subplot(211)
# g = sns.countplot('month', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Counting Months ", fontsize=20)
# g.set_xlabel("Months", fontsize=15)
# g.set_ylabel("Count", fontsize=15)

# plt.subplot(212)
# g1 = sns.lvplot(x='month', y='like_rate', data=df_yout, palette="Set1")
# g1.set_xticklabels(g.get_xticklabels(),rotation=45)
# g1.set_title("Like Rate by Month", fontsize=20)
# g1.set_xlabel("Months", fontsize=15)
# g1.set_ylabel("Like Rate(log)", fontsize=15)

# plt.subplots_adjust(hspace = 0.5, top = 0.9)

# plt.show()

## Let's extract the hour value of datetime'

In [ ]:
# separates date and time into two columns from 'publish_time' column
df_yout.insert(4, 'publish_date', df_yout['publish_time'].dt.date)
df_yout['publish_time'] = df_yout['publish_time'].dt.time
df_yout[['hour','min','sec']] = df_yout['publish_time'].astype(str).str.split(':', expand=True).astype(int)

## Knowning the Hour feature

In [ ]:
print("TOP 5 Hour values")
print(df_yout['hour'].value_counts()[:5])

# plt.figure(figsize = (14,9))

# plt.subplot(211)
# g = sns.countplot('hour', data=df_yout, palette="Set1")
# g.set_xticklabels(g.get_xticklabels(),rotation=45)
# g.set_title("Counting Hour ", fontsize=20)
# g.set_xlabel("Hours", fontsize=15)
# g.set_ylabel("Count", fontsize=15)

# plt.subplot(212)
# g1 = sns.lvplot(x='hour', y='like_rate', data=df_yout, palette="Set1")
# g1.set_xticklabels(g.get_xticklabels(),rotation=45)
# g1.set_title("Like Rate by Hour", fontsize=20)
# g1.set_xlabel("Hours", fontsize=15)
# g1.set_ylabel("Like Rate(log)", fontsize=15)

# plt.subplots_adjust(hspace = 0.5, top = 0.9)

# plt.show()

Very interesting distribuition by hour

# Votesup and stay Tuned because I will continue this analysis